In [1]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [4]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [5]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2024-01-01'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2024-01-01
Área de Interesse: Bioma Cerrado


In [6]:
# ==============================================================================
# PASSO 2: COLETA DA ÁREA QUEIMADA (MAPBIOMAS)
# ==============================================================================

# 1. Carregamos o asset global como uma única ee.Image, SEM NENHUM CORTE.
burned_area_image_global = ee.Image('projects/mapbiomas-public/assets/brazil/fire/collection4/mapbiomas_fire_collection4_monthly_burned_v1')

# --- TESTE DE VERIFICAÇÃO SEGURO ---
# Agora fazemos as perguntas ao objeto global e simples.
print("Iniciando verificação segura no objeto global...")

# Nomes de todas as bandas do objeto global.
band_names_server = burned_area_image_global.bandNames()

# Pedimos ao servidor para nos enviar o NÚMERO TOTAL de bandas.
num_bands = band_names_server.size().getInfo()
print(f"O objeto global é uma imagem única com {num_bands} bandas.")

# Pedimos ao servidor para "fatiar" a lista e nos enviar APENAS OS 5 PRIMEIROS NOMES.
last_5_bands = band_names_server.slice(35, 40).getInfo()
print("Exemplo dos ultimos 5 nomes de banda:", last_5_bands)
print("Verificação concluída com sucesso.")

# ------------------------------------------------------------------------------

# 2. Objeto cortado para uso futuro
burned_area_cerrado = burned_area_image_global.clip(cerrado_aoi)

print("\nObjeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.")

Iniciando verificação segura no objeto global...
O objeto global é uma imagem única com 40 bandas.
Exemplo dos ultimos 5 nomes de banda: ['burned_monthly_2020', 'burned_monthly_2021', 'burned_monthly_2022', 'burned_monthly_2023', 'burned_monthly_2024']
Verificação concluída com sucesso.

Objeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.


In [31]:
# --- Passo 1: Carregar a Coleção MapBiomas Fogo (Anual) ---

mapbiomasFogo = ee.Image('projects/mapbiomas-public/assets/brazil/fire/collection4/mapbiomas_fire_collection4_annual_burned_v1')



# --- Passo 2: Definir os Anos de Interesse ---

anoAnterior = 2013  # O ano que usaremos como "máscara"

anoAlvo = 2014      # O ano que queremos "limpar"



# --- Passo 3: Selecionar as Imagens para Cada Ano ---



# Pega a imagem de 2013.

queimada2013 = mapbiomasFogo.select('burned_area_2013').clip(cerrado_aoi)



# Pega a imagem de 2014.

queimada2014 = burned_area_image_global.select('burned_monthly_2014').clip(cerrado_aoi)

mascaraJaneiro2024 = queimada2014.eq(1)



# --- Passo 4: A "Limpeza" dos Dados (A Lógica) ---

# Queremos pixels que:

# (Queimaram em 2014 > 0) E (NÃO queimaram em 2013 == 0)



# Máscara binária: 1 para pixels queimados em 2014

mascaraQueimada2014 = queimada2014.eq(1)



# Máscara binária: 1 para pixels NÃO queimados em 2013

mascaraNaoQueimadaJaneiro2013 = queimada2013.eq(0)



# Combina as duas condições. O resultado é 1 (True) APENAS se AMBAS forem verdadeiras.

novasQueimadasJaneiro2014 = mascaraQueimada2014.And(mascaraNaoQueimadaJaneiro2013)



# --- Passo 5: Visualizar os Resultados com geemap ---



# Criar um mapa interativo

Map = geemap.Map()



# Centraliza o mapa na área que eu quiser

Map.setCenter(-44.616884, -8.623232, 11) # Longitude, Latitude, Zoom



# Define parâmetros de visualização

visParamsBinario = {

  'min': 0,

  'max': 1,

  'palette': ['#FFFFFF', '#000000'] # 0=transparente, 1=vermelho

}



visParamsOriginal = {

  'min': 1,   # Mês 1 (Jan)

  'max': 12,  # Mês 12 (Dez)

  'palette': ['#8C2D04', '#79258A', '#006EFF', '#00FF1A', '#3D0000', '#6E02FA', '#FFFFD4']

}





# --- Adicionar Camadas ao Mapa ---



# Adiciona a camada ORIGINAL de 2013 (para referência)

Map.addLayer(
    queimada2013.selfMask(),
    {'palette': 'orange'}, # Apenas uma cor para o '1'
    'Total Queimado 2013 (Anual)',
    False # Deixar desligado por padrão (ajuda a não estourar memória)
)



# Adiciona a camada ORIGINAL de 2014 (os dados "sujos")

Map.addLayer(queimada2014.updateMask(mascaraQueimada2014), visParamsOriginal, 'Total Queimado 2014 (Original/Sujo)')

# Adiciona a camada RESULTANTE (os dados "limpos")

# .selfMask() oculta os pixels 0, mostrando apenas os 1

Map.addLayer(
    novasQueimadasJaneiro2014.selfMask(),  # <--- A MÁGICA ESTÁ AQUI
    {'palette': '#000000'},                 # <--- Pinta o '1' de preto
    'Novas Queimadas em Janeiro de 2014 (Limpo)'
)

# Adiciona um controle de camadas para ligar/desligar

Map.addLayerControl()

# --- Exibir o Mapa ---

# No Jupyter/Colab, basta ter o objeto 'Map' como a última linha da célula

Map

Map(center=[-8.623232, -44.616884], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [19]:
# ==============================================================================
# PASSO 2.1 : DESCOMPACTAR DADOS E CRIAR COLEÇÃO MENSAL
# ==============================================================================

# Lista de anos do nosso dataset
years = ee.List.sequence(2013, 2024)
# Lista de meses
months = ee.List.sequence(1, 12)

# Usamos um.map() aninhado para criar uma imagem binária para cada mês de cada ano
# Isso transforma 40 bandas em 480 imagens
def create_monthly_images(year):
    # Seleciona a banda do ano
    annual_band = burned_area_cerrado.select(ee.String('burned_monthly_').cat(ee.Number(year).int()))

    def create_monthly_mask(month):
        # Cria a imagem binária (1 ou 0) para aquele mês específico
        monthly_mask = annual_band.eq(ee.Number(month)).selfMask()

        # Define a data correta para a imagem.
        date = ee.Date.fromYMD(year, month, 1)
        return monthly_mask.set('system:time_start', date.millis())

    return months.map(create_monthly_mask)

# O.flatten() transforma a lista de listas em uma única coleção de 480 imagens
monthly_burned_collection = ee.ImageCollection(years.map(create_monthly_images).flatten())

print("Coleção mensal de 480 imagens criada com sucesso no servidor.")

Coleção mensal de 480 imagens criada com sucesso no servidor.


In [20]:
# ==============================================================================
# PASSO 2.1.1 (REVISADO E CORRIGIDO): COLETA E TRANSFORMAÇÃO DOS DADOS DE FOCOS DE CALOR
# ==============================================================================

# 1. Carregamos a coleção de IMAGENS diárias de fogo do MODIS (Produto MOD14A1).
daily_fire_images = ee.ImageCollection('MODIS/061/MOD14A1') \
.filterDate(START_DATE, END_DATE) \
.filterBounds(cerrado_aoi) \
.select('FireMask')

# 2. Função para transformar os pixels de fogo de uma imagem diária em pontos (features).
# Esta função inclui todas as correções que fizemos para os erros de memória e de tipo.
def raster_to_points(image):
    # Extrai a data da imagem para anexar ao ponto.
    date = image.date().format('YYYY-MM-dd')

    # Cria uma máscara binária (1) para pixels de fogo (valores 7, 8, ou 9)
    fire_pixels = image.gte(7)

    # Torna todos os pixels 0 (sem fogo) transparentes para otimização
    fire_pixels = fire_pixels.selfMask()

    # Converte os pixels de fogo restantes em pontos (vetores).
    fire_points = fire_pixels.reduceToVectors(
        geometry=cerrado_aoi,
        scale=1000,                # Resolução do MODIS (1km)
        geometryType='centroid',
        eightConnected=False
    )

    # Bloco if/else que roda no servidor
    conditional_result = ee.Algorithms.If(
        fire_points.size(), # Se algum ponto de fogo foi encontrado...
        fire_points.map(lambda point: point.set('ACQ_DATE', date)), #... anexa a data a eles.
        ee.FeatureCollection([])
    )

    # "Casta" o resultado de volta para FeatureCollection para evitar erros de tipo.
    return ee.FeatureCollection(conditional_result)

# 3. Aplicamos a função a cada imagem da coleção e "achatamos" o resultado.
# O.map() aplica a função. O.flatten() junta a lista de listas em uma só.
active_fires_collection = daily_fire_images.map(raster_to_points).flatten()

print("Objeto 'active_fires_collection' definido com sucesso no servidor.")
print("Esta célula apenas define a 'receita'. A computação pesada ocorrerá na exportação.")

Objeto 'active_fires_collection' definido com sucesso no servidor.
Esta célula apenas define a 'receita'. A computação pesada ocorrerá na exportação.


In [21]:
# ==============================================================================
# PASSO 2.2 (NOVO): FILTRAR PROPAGAÇÃO E ISOLAR IGNIÇÕES VERDADEIRAS
# ==============================================================================

# Definimos nossa data de início de FILTRAGEM para ser um mês antes do nosso START_DATE
FILTER_START_DATE = '2013-12-01'

# Filtramos a coleção de 480 meses para conter apenas o período que nos interessa.
# Isso reduz a carga de processamento em 75% e EVITA o erro de Jan/1985.
monthly_collection_filtered = monthly_burned_collection.filterDate(FILTER_START_DATE, END_DATE)

# Define um filtro de tempo. 'maxDifference' é usado para encontrar imagens vizinhas no tempo.
max_diff = ee.Filter.maxDifference(
    # A diferença máxima é de 32 dias em milissegundos.
    # Usamos 32 dias para garantir que pegamos o mês anterior, mesmo em meses com 31 dias.
    difference=32 * 24 * 60 * 60 * 1000,
    # Campos que o filtro deve comparar: a data de início de uma imagem...
    leftField='system:time_start',
    #...com a data de início de outra imagem.
    rightField='system:time_start'
)

# Define uma "Junção" (Join) do GEE. É uma operação de banco de dados otimizada.
# 'saveFirst' diz ao GEE: "para cada imagem, encontre a primeira (e única) imagem
# que corresponde ao filtro 'max_diff' e salve-a".
# 'matchKey' é o nome da nova propriedade que armazenará a imagem encontrada.
save_previous_join = ee.Join.saveFirst(matchKey='previous_month')

# Aplica a junção. Juntamos a coleção com ela mesma.
# 'primary' é a coleção principal (Mês M).
# 'secondary' é a coleção onde ele procura correspondências (Mês M-1).
# 'condition' é a regra de filtro que definimos (32 dias antes).
collection_with_previous = save_previous_join.apply(
    primary=monthly_burned_collection,
    secondary=monthly_burned_collection,
    condition=max_diff
)

# Define uma função que será aplicada (mapeada) a cada imagem da coleção resultante da junção.
def filter_propagation(current_month_image):
    # Converte a entrada (que pode ser um objeto genérico) em um objeto ee.Image.
    current_month_image = ee.Image(current_month_image)

    # Pega a imagem do mês anterior que foi anexada pela junção na propriedade 'previous_month'.
    previous_month_image = ee.Image(current_month_image.get('previous_month'))

    # ESTA É A LÓGICA DE VIZINHANÇA ("ENGORDAR" A CICATRIZ)
    #.focal_max() é uma operação de "kernel" ou vizinhança.
    # 'radius=1, kernelType='square'' define uma janela de 3x3 pixels (8 vizinhos + o central).
    #.focal_max() faz com que, se *qualquer* pixel na janela for 1, o pixel central
    # do resultado também se torne 1. O efeito é "dilatar" ou "engordar" a cicatriz
    # do mês anterior em 1 pixel (30m) em todas as direções.
    dilated_previous_scar = previous_month_image.focal_max(radius=1, kernelType='square', units='pixels')

    # ESTA É A LÓGICA DE SUBTRAÇÃO (A SUA IDEIA EM CÓDIGO)
    #.Not() inverte a imagem "engordada". Pixels da frente de fogo se tornam 0, o resto 1.
    #.And() é uma operação lógica "E". O resultado só é 1 se o pixel for 1 em AMBAS as imagens.
    # A lógica é: (Pixel queimou no Mês M) E (Pixel NÃO estava tocando a frente de fogo do Mês M-1)
    true_ignition_mask = current_month_image.And(dilated_previous_scar.Not())

    # Retorna a máscara de ignição verdadeira, torna os 0s transparentes (.selfMask())
    # e copia as propriedades (como a data) da imagem original.
    return true_ignition_mask.selfMask().copyProperties(current_month_image)

# Aplica a função de filtragem a todas as imagens da coleção.
#.flatten() é usado para remover com segurança as imagens que retornaram nulo
# (especificamente, a primeira imagem, de 1985, que não tinha um "mês anterior").
true_ignitions_collection = ee.ImageCollection(collection_with_previous.map(filter_propagation).flatten())

print("Filtro de propagação criado. 'true_ignitions_collection' agora contém apenas pixels de ignição verdadeira.")

Filtro de propagação criado. 'true_ignitions_collection' agora contém apenas pixels de ignição verdadeira.


In [18]:
# ==============================================================================
# PASSO 2.3 (SUBSTITUI O 2.1.3): EXECUTAR A JUNÇÃO FINAL
# ==============================================================================

# Define o ano que queremos processar e exportar.
YEAR_TO_PROCESS = 2022

# Filtra nossa NOVA coleção de ignições verdadeiras para o ano de 2022.
true_ignitions_2022 = true_ignitions_collection.filterDate(f'{YEAR_TO_PROCESS}-01-01', f'{YEAR_TO_PROCESS}-12-31')

# --- 2. Identificar e vetorizar as MANCHAS DE IGNIÇÃO VERDADEIRA ---
#.mosaic() achata as 12 imagens mensais de 2022 em um único mapa de pixels de ignição verdadeira para o ano.
true_ignitions_image_2022 = true_ignitions_2022.mosaic()

#.connectedComponents() agrupa os pixels de ignição em manchas com IDs únicos.
fire_patches_image = true_ignitions_image_2022.connectedComponents(ee.Kernel.plus(1), 1024)

#.reduceToVectors() converte as manchas de pixels (raster) em polígonos (vetor).
# Esta é a operação pesada que agora está sendo aplicada a dados limpos.
fire_polygons = fire_patches_image.reduceToVectors(
    geometry=cerrado_aoi,
    scale=TARGET_RESOLUTION,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='patch_id'
)

# --- 3. Filtrar os focos de calor para o mesmo ano ---
# Pega nossa coleção de focos de calor do MODIS (do Passo 2.1.1 anterior) e filtra para 2022.
active_fires_2022 = active_fires_collection.filterDate(f'{YEAR_TO_PROCESS}-01-01', f'{YEAR_TO_PROCESS}-12-31')

# --- 4. Realizar a Junção Espacial (Lógica idêntica à anterior) ---
# Define a regra: encontrar pontos DENTRO de polígonos.
spatial_filter = ee.Filter.withinDistance(distance=0, rightField='.geo', leftField='.geo')
# Define a ação: salvar TODOS os pontos correspondentes em uma lista chamada 'matches'.
save_all_join = ee.Join.saveAll(matchesKey='matches')
# Aplica a junção: para cada polígono, encontra e anexa os pontos de fogo que estão dentro dele.
intersecting_polygons = save_all_join.apply(fire_polygons, active_fires_2022, spatial_filter)

# --- 5. Mapear sobre o resultado da junção ---
# Define uma função que será aplicada a cada polígono que teve uma correspondência.
def find_first_ignition(polygon_with_matches):
    # Pega a lista de pontos da propriedade 'matches' e a transforma em uma coleção.
    matched_points = ee.FeatureCollection(ee.List(polygon_with_matches.get('matches')))
    # Ordena essa pequena coleção de pontos por data...
    #...e pega o primeiro (.first()), que é o foco de calor mais antigo (a ignição).
    first_point = matched_points.sort('ACQ_DATE').first()
    # Retorna apenas o ponto.
    return first_point.copyProperties(polygon_with_matches)

# Aplica a função e descarta os polígonos que não tiveram nenhum foco de calor (dropNulls=True).
ignition_points_raw = intersecting_polygons.map(find_first_ignition, dropNulls=True)

# --- 6. Formatar os resultados ---
# Define uma função de limpeza final.
def format_ignition_point(feature):
    # Cria uma nova feição...
    return ee.Feature(
        feature.geometry(), #...usando a geometria do ponto...
        { #...e selecionando apenas a propriedade 'acq_date' que nos interessa.
            'acq_date': feature.get('ACQ_DATE')
        }
    )

# Aplica a função de formatação.
ignition_points_2022 = ignition_points_raw.map(format_ignition_point)

# Imprime mensagens de status. Estas linhas rodam instantaneamente.
print(f"Lógica de junção espacial para {YEAR_TO_PROCESS} (com filtro de propagação) definida.")
print("O objeto 'ignition_points_2022' está pronto para ser exportado.")

# --- 7. EXPORTAR (O antigo Passo 2.1.4) ---
# Define o nome do arquivo de saída.
output_filename = f'TRUE_ignition_points_cerrado_{YEAR_TO_PROCESS}'
# Define a tarefa de exportação assíncrona.
task = ee.batch.Export.table.toDrive(
    collection=ignition_points_2022,
    description=output_filename,
    folder='/content/drive/MyDrive/Projeto_Incendio/data/raw', # Nome da pasta no seu Google Drive
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# Inicia a tarefa no servidor.
task.start()

print("Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.")
print(f"O arquivo '{output_filename}.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.")

Lógica de junção espacial para 2022 (com filtro de propagação) definida.
O objeto 'ignition_points_2022' está pronto para ser exportado.
Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.
O arquivo 'TRUE_ignition_points_cerrado_2022.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.


In [ ]:
# ==============================================================================
# PASSO 4: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 4.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi)

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 4.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 4.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').clip(cerrado_aoi)
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- 4.4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2').clip(cerrado_aoi)

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87648 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3631 imagens de LST (diárias).
Dados de elevação e declive carregados.
Dados de uso e cobertura do solo carregados.


In [ ]:
# ==============================================================================
# PASSO 5: EXPORTAR RÓTULOS PARA O DRIVE
# ==============================================================================

# Define o nome do arquivo e a tarefa de exportação.
output_filename = f'ignition_points_cerrado_{YEAR_TO_PROCESS}'
task = ee.batch.Export.table.toDrive(
    collection=ignition_points_2022,
    description=output_filename,
    folder='Projeto_Incendios/data/raw',  # A pasta que criamos no seu Google Drive
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# Inicia a tarefa de exportação.
task.start()

print("Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.")
print(f"O arquivo '{output_filename}.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.")